# Importing Libraries

In [7]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

import imageio
import subprocess
import json
import torch

from transformers import AutoProcessor, AutoModelForVision2Seq
from tqdm import tqdm  

In [8]:
df = pd.read_parquet("test-00000-of-00001.parquet")

mask = df["question_prompt"].str.contains("E. None of the above", na=False)

# Append "E. None of the above" on a new line to "question"
df.loc[mask, "question"] = df["question"] + "\nE. None of the above"

# Remove only "E. None of the above" from "question_prompt" while keeping the rest
df.loc[mask, "question_prompt"] = df["question_prompt"].str.replace("E. None of the above", "", regex=False).str.strip()

df

,qid,video_id,question_type,capability,question,duration,question_prompt,answer,youtube_url
0,0008-0,sj81PWrerDk,Primary Open-ended Question,Plot Attribute (Montage),What is the difference between the action of t...,8.85,Please state your answer with a brief explanat...,,https://www.youtube.com/shorts/sj81PWrerDk
1,0008-1,sj81PWrerDk,Paraphrased Open-ended Question,Plot Attribute (Montage),Can you describe how the actions of the last p...,8.85,Please state your answer with a brief explanat...,,https://www.youtube.com/shorts/sj81PWrerDk
2,0008-2,sj81PWrerDk,Correctly-led Open-ended Question,Plot Attribute (Montage),Did the last person open the bottle without us...,8.85,Please state your answer with a brief explanat...,,https://www.youtube.com/shorts/sj81PWrerDk
3,0008-3,sj81PWrerDk,Wrongly-led Open-ended Question,Plot Attribute (Montage),Did the last person in the video open the bott...,8.85,Please state your answer with a brief explanat...,,https://www.youtube.com/shorts/sj81PWrerDk
4,0008-7,sj81PWrerDk,Multiple-choice Question with a Single Correct...,Plot Attribute (Montage),How does the last person in the video open the...,8.85,Select one best answer to the above multiple-c...,,https://www.youtube.com/shorts/sj81PWrerDk
...,...,...,...,...,...,...,...,...,...
1495,1344-0,eLJNa61S4RE,Primary Open-ended Question,Character Reaction Causality,"In this video, why does the child smile after ...",18.35,Please state your answer with a brief explanat...,,https://www.youtube.com/shorts/eLJNa61S4RE
1496,1344-1,eLJNa61S4RE,Paraphrased Open-ended Question,Character Reaction Causality,What causes the child to smile after receiving...,18.35,Please state your answer with a brief explanat...,,https://www.youtube.com/shorts/eLJNa61S4RE
1497,1344-2,eLJNa61S4RE,Correctly-led Open-ended Question,Character Reaction Causality,Does the child smile after the injection becau...,18.35,Please state your answer with a brief explanat...,,https://www.youtube.com/shorts/eLJNa61S4RE
1498,1344-3,eLJNa61S4RE,Wrongly-led Open-ended Question,Character Reaction Causality,Is the child's smile after the injection due t...,18.35,Please state your answer with a brief explanat...,,https://www.youtube.com/shorts/eLJNa61S4RE


In [ ]:
filtered_df = df[df["video_id"]=="_a6lFCUYTA4"]
filtered_df

In [ ]:
unique_question_types = df["question_type"].unique().tolist()
print(unique_question_types)

In [ ]:
def get_video_metadata(video_path):
    cmd = [
        "ffprobe",
        "-v", "error",
        "-select_streams", "v:0",
        "-show_entries", "stream=nb_frames,r_frame_rate,duration,width,height",
        "-of", "json",
        video_path
    ]
    
    result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    return json.loads(result.stdout)

video_path = "Benchmark-AllVideos-HQ-Encoded-challenge/_a6lFCUYTA4.mp4"

metadata = get_video_metadata(video_path)
frame_count = int(metadata['streams'][0]['nb_frames'])  # safely extracted

reader = imageio.get_reader(video_path)
frame_indices = np.linspace(0, frame_count - 1, num=50, dtype=int)

for i in frame_indices:
    frame = reader.get_data(i)  # Get specific frame by index
    plt.imshow(frame)
    plt.title(f"Sampled Frame {i}")
    plt.axis("off")
    plt.show()

# Sliding Window for cropping

In [ ]:
# def sliding_window_crop(image: Image.Image, patch_size=512, stride=256):
#     """
#     Generate a list of crops from the image using a sliding window.
#     """
#     width, height = image.size
#     crops = []

#     for y in range(0, height - patch_size + 1, stride):
#         for x in range(0, width - patch_size + 1, stride):
#             crop = image.crop((x, y, x + patch_size, y + patch_size))
#             crops.append(crop)
    
#     return crops

In [ ]:
# all_crops = []
# for frame in frame_indices:
#     pil_img = Image.fromarray(frame.asnumpy())
#     crops = sliding_window_crop(pil_img, patch_size=512, stride=256)
#     all_crops.extend(crops)

# Testing Model

In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

processor = AutoProcessor.from_pretrained("HuggingFaceTB/SmolVLM2-256M-Video-Instruct")
model = AutoModelForVision2Seq.from_pretrained(
    "HuggingFaceTB/SmolVLM2-256M-Video-Instruct",
    torch_dtype=torch.bfloat16 if DEVICE == "cuda" else torch.float32,
).to(DEVICE)

In [ ]:
def sample_frames(video_path, num_frames=100):
    reader = imageio.get_reader(video_path)
    total_frames = len(reader)
    indices = np.linspace(0, total_frames - 1, num=num_frames, dtype=int)
    frames = [Image.fromarray(reader.get_data(i)) for i in indices]
    return frames

# Store results in a nested dict
results = {}

unique_videos = df["video_id"].unique().tolist()

for video in unique_videos:

    temp_df = df[df["video_id"]==unique_videos]
    question_list = temp_df["question"].unique().tolist()

    for question in question_list:
        temp_temp_df = temp_df[temp_df["question"]==question]

        video_path = "Benchmark-AllVideos-HQ-Encoded-challenge/" + temp_temp_df["video_id"]
        question = temp_temp_df["question"]
        question_prompt = temp_temp_df["question_prompt"]

        full_prompt = f'{question}\n{question_prompt}'
        frames = sample_frames(video_path)

        inputs = processor(
            text=full_prompt,
            images=frames,
            return_tensors="pt",
            padding=True,
            truncation=True
        ).to(DEVICE)

        with torch.no_grad():
            generated_ids = model.generate(**inputs, max_new_tokens=100)

        output_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
        print(f'Video: {video}')
        print(f'Question: {full_prompt}')     
        print(f'Answer: {output_text}')   
        temp_temp_df["answer"] = output_text

In [ ]:
# import torch
# import pandas as pd
# import numpy as np
# import cv2
# import os
# from PIL import Image
# from transformers import AutoProcessor, AutoModelForVision2Seq

# # Select device (CUDA → MPS → CPU)
# if torch.cuda.is_available():
#     DEVICE = "cuda"
# elif torch.backends.mps.is_available():
#     DEVICE = "mps"
# else:
#     DEVICE = "cpu"

# # Force CPU if needed
# DEVICE = "cpu"

# # Define video folder
# video_folder = "Benchmark-AllVideos-HQ-Encoded-challenge"

# # Load model and processor
# processor = AutoProcessor.from_pretrained("HuggingFaceTB/SmolVLM-500M-Instruct")
# model = AutoModelForVision2Seq.from_pretrained(
#     "HuggingFaceTB/SmolVLM-500M-Instruct",
#     torch_dtype=torch.float16 if DEVICE == "mps" else torch.bfloat16,
#     _attn_implementation="flash_attention_2" if DEVICE == "cuda" else "eager",
# ).to(DEVICE)

# # Function to extract video frames
# def extract_frames(video_path, max_frames=5, target_size=384):  # Reduce max frames to 5
#     cap = cv2.VideoCapture(video_path)
#     if not cap.isOpened():
#         raise ValueError(f"Could not open video: {video_path}")

#     total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
#     fps = int(cap.get(cv2.CAP_PROP_FPS))
    
#     # Extract frames at 1 FPS
#     frame_indices = list(range(0, total_frames, fps))
    
#     # If too many frames, evenly sample max_frames
#     if len(frame_indices) > max_frames:
#         frame_indices = [frame_indices[i] for i in np.linspace(0, len(frame_indices) - 1, max_frames, dtype=int)]
    
#     frames = []
#     for frame_idx in frame_indices:
#         cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
#         ret, frame = cap.read()
#         if ret:
#             frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert to RGB
#             pil_image = Image.fromarray(frame).resize((target_size, target_size))
#             frames.append(pil_image)

#     cap.release()
#     return frames

# for file in os.listdir(video_folder):
#     video_path = os.path.join(video_folder, file)

#     video_name = os.path.splitext(file)[0]  # Remove file extension
#     video_name = video_name.lstrip("_")  # Remove leading underscore

#     temp_df = df[df["youtube_url"].str.contains(video_name, na=False)]
#     print(f"Matched rows: {len(temp_df)}")

#     for index, row in temp_df.iterrows():
#         # Extract frames from video
#         max_frames = 5  # Limit to 5 images
#         frames = extract_frames(video_path, max_frames)

#         # Ensure the prompt contains the same number of `<image>` tokens as the number of frames
#         image_tokens = "<image> " * len(frames)

#         # Truncate the question prompt if too long
#         MAX_TEXT_LENGTH = 500  # Set a max length for text input
#         question_text = f"{row['question']}\n{row['question_prompt']}"[:MAX_TEXT_LENGTH]

#         # Construct the input question with `<image>` tokens
#         full_prompt = f"{image_tokens}\n{question_text}"

#         # Prepare inputs WITHOUT using chat template
#         inputs = processor(
#             text=[full_prompt],  # Pass the question with <image> tokens
#             images=frames,  # Pass extracted video frames
#             return_tensors="pt",
#             truncation=False,  # Prevent unwanted truncation
#             max_length=2048,  # Enforce max token length
#         )

#         # Debug tensor shapes before feeding into model
#         print(f"Input IDs Shape: {inputs['input_ids'].shape}")
#         print(f"Pixel Values Shape: {inputs['pixel_values'].shape}")

#         # Move to correct device and ensure float16 conversion for MPS
#         if DEVICE == "mps":
#             inputs = {
#                 key: (value.to(DEVICE).to(torch.float16) if value.dtype in [torch.float, torch.float32] else value.to(DEVICE))
#                 for key, value in inputs.items()
#             }
#         else:
#             inputs = {key: value.to(DEVICE) for key, value in inputs.items()}

#         # Generate output
#         generated_ids = model.generate(
#             **inputs,
#             max_new_tokens=150,  # Reduce max generation length
#             repetition_penalty=1.2,  # Avoid repeating words
#         )

#         # Decode and clean response
#         response = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
#         response = response.replace(question_text, "").strip()  # Remove repeated question text

#         # Print final output
#         print(f"Question: {row['question']}\n{row['question_prompt']}")
#         print(f"Answer: {response}\n")
